In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the CSV file with boundary conditions 
file_path = 'BoundaryConditions.csv' 
try:
    df = pd.read_csv(file_path, header=1)
    print(f"Archivo '{file_path}' correctly load.")
except Exception as e:
    print(f"Error al cargar '{file_path}': {e}")
    df = pd.DataFrame() 

Archivo 'BoundaryConditions.csv' correctly load.


In [3]:
df_filtered = pd.DataFrame() 
if not df.empty:
    id_col = 'Product ID'  
    attr_col = 'Attribute' 
    if id_col in df.columns and attr_col in df.columns:
        condition = (
            df[id_col].isin(['Total', '21A', '22B', '23C']) &
            (df[attr_col] == 'Available Capacity')
        )
        df_filtered = df.loc[condition].copy()
        print(f"Files filtered by 'Available Capacity'. Shape: {df_filtered.shape}")
    else:
        print(f"Error: Columns weren't find '{id_col}' o '{attr_col}'. Verify the load.")

Files filtered by 'Available Capacity'. Shape: (4, 445)


In [4]:
if not df_filtered.empty:
    df_filtered.drop_duplicates(inplace=True)

In [5]:
if not df_filtered.empty:
    value_columns = df_filtered.columns[2:]
    for col in value_columns:
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

    nan_count = df_filtered[value_columns].isnull().sum().sum()
    if nan_count > 0:
        print(f"Alerta: {nan_count} NaNs generados durante conversión numérica.")
        # Consider fill the Nans values, ej:
        # df_filtered[value_columns] = df_filtered[value_columns].fillna(0)

In [6]:
df_long_capacity = pd.DataFrame() 
if not df_filtered.empty:
    try:
        id_col_name = df_filtered.columns[0]
        attr_col_name = df_filtered.columns[1]
        value_vars = df_filtered.columns[2:] 

        df_to_melt = df_filtered.rename(columns={
            id_col_name: 'Product_ID',
            attr_col_name: 'Constraint_Type'
        })

        df_long_capacity = pd.melt(
            df_to_melt,
            id_vars=['Product_ID', 'Constraint_Type'],
            value_vars=value_vars,
            var_name='Week_Original',    
            value_name='Capacity_Value'
        )
        print(f"Restructured Dataframe to long shape. Final form: {df_long_capacity.shape}")
    except Exception as e:
        print(f"Error durante el melt: {e}")

Restructured Dataframe to long shape. Final form: (1772, 4)


In [7]:
if not df_long_capacity.empty:
    print("\n--- DataFrame head processed ---")
    print(df_long_capacity.head())
    print("\n--- DataFrame Tail processed ---")
    print(df_long_capacity.tail())
    print(df_long_capacity.info()) 


--- DataFrame head processed ---
  Product_ID     Constraint_Type Week_Original  Capacity_Value
0      Total  Available Capacity         WW_09         11037.0
1        21A  Available Capacity         WW_09          5155.0
2        22B  Available Capacity         WW_09          4000.0
3        23C  Available Capacity         WW_09          1882.0
4      Total  Available Capacity         WW_10         10791.0

--- DataFrame Tail processed ---
     Product_ID     Constraint_Type Week_Original  Capacity_Value
1767        23C  Available Capacity       WW_34.8           500.0
1768      Total  Available Capacity       WW_35.8          8000.0
1769        21A  Available Capacity       WW_35.8          5000.0
1770        22B  Available Capacity       WW_35.8          2500.0
1771        23C  Available Capacity       WW_35.8           500.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1772 entries, 0 to 1771
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  -

In [8]:
output_file = "BoundaryConditions_AvailableCapacity_Long.csv"
if not df_long_capacity.empty:
    df_long_capacity.to_csv(output_file, index=False)
    print(f"Data saved in '{output_file}'")
else:
    print("El DataFrame final está vacío, no se guardó ningún archivo.")

Data saved in 'BoundaryConditions_AvailableCapacity_Long.csv'
